# Pré-processamento e segmentação

Esta é uma etapa importante do processamento de dados, que envolve vários conceitos. No pré-processamento, os dados podem ser "limpados", padronizados e transformados.

- *Limpar* os dados refere-se à remoção ou correção de dados que estejam incompletos, corrompidos ou imprecisos.
- *Padronizar* os dados refere-se à remoção de *outliers*, além de deixar todas as possíveis mensurações na mesma escala e unidade, além de normalizá-las quando necerrário.
- *Transformar* os dados refere-se à aplicação dos dados em um formato que favoreça futuras extrações de características ou análises.

In [1]:
# carregamento e preparação dos dados
%run "load_dataset.ipynb"

(20000, 2)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)


## Filtragem dos dados

Ao coletar dados EMG, estes tendem a vir com uma interferência forte externa. A interferência ocorre na frequência da rede elétrica. No Brasil a rede elétrica possui uma frequência de 60 Hz, gerando uma interferência em volta da mesma frequência. Em outros países, como os pertencentes à Europa, a frequência da rede elétrica é de 50 Hz. A interferência precisa ser removida dos dados, utilizando um filtro de negação de faixa "*notch*". Já foi dita que a base de dados utilizada neste tutorial já passou por este processo antes de ser disponibilizada pelo autor. Porém, vamos analisar o sinal e verificar se ainda existem resquícios, e então aplicaremos aqui os filtros necessários.

In [2]:
# carregamento dos filtros temporais
%run "filters.py"

In [3]:
# aplicação de filtros temporais

data_filtered = butter_notch(data, 60)
data_filtered = butter_highpass(data_filtered, 5)
data_filtered = butter_lowpass(data_filtered, 50)

data_filtered.shape

(10, 6, 2, 20000)

**Desafio 1**: Como vemos o resultado de `data_filtered`, existe apenas um conjunto de dados para cada movimento. Modifique o código de forma que as seis diferentes tentativas fiquem na mesma dimensão do conjunto. Por exemplo, ao invés do shape resultar em: (10, 6, 2, 20000), queremos que o shape final fique da seguinte forma: (60, 2, 20000). Isto faz com que a quantidade de ensaios não perca seu propósito e seja incorporado aos dados de processamento do problema.

## Segmentação dos dados

As características normalmente são extraídas sobre pequenos segmentos de tamanho fixo dos dados, não no dado como um todo. Estes pequenos segmentos são chamados de janelas. A técnica de separar os dados em janelas, recebe o nome de janela deslizante (*sliding window*) e é uma forma de segmentação de dados. Uma boa prática, é definir um passo para essa janela de forma que haja uma sopreposição de dados, para que informações da lacuna de uma janela e outra não sejam perdidas. Na imagem a seguir podemos observar um sinal EMG. Abaixo dele a representação de janelas: *W1*, *W2*, *W3*... Repare que entre as janelas, há uma sobreposição de tamanho *T*. 

Nesta base de dados, cada ensaio tem duração de 5 segundos. Se utilizadas janelas com tamanho de 250 ms, resultará em 20 janelas. Ao aplicar uma sobreposição de ~128ms, ficamos com 41 janelas de ~122ms.

Algumas características de EMG trabalham com o dado no domínio da frequência. Quando tais características são aplicadas, é necessário tranformar o dado para o domínio da frequência, utilizando o método de transformação de domínio `STFT`, do inglês *Short-time Fourier transform* (Transformada de Fourier de curto termo). O código divide o dado em segmentos, tanto no domínio do tempo quanto no domínio da frequência.

In [4]:
from scipy.signal import stft

step = 470
segment = 1024
data = data_filtered.reshape(60, 2, 20000)
print('', data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * step

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

# Janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=4000, nperseg=1024, noverlap=512)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')


 (60, 2, 20000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (60, 2, 41, 1024) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (60, 2, 41, 513) - (classes+ensaios, canais, janelas, linhas)


**Desafio 2**: Organize os dados de forma que a quantidade de ensaios tenham sido reelaboradas.

**Desafio 3**: Implemente uma função para criar os segmentos que receba como parâmetro os dados que definem o tamanho e o passo da janela. Diferentes tamanhos de janelas e de sobreposições influenciam na extração de características. Esta função será útil para testar diversas variações. **Dica**: observe que os dados para tamanho e passo da janela são diferentes entre o domínio do tempo e da frequência. Pode ser necessário a implementação de funções auxiliares e/ou a inclusão de mais parâmetros.